# Facial Recognition System

In [1]:
from getpass import getpass
uplink_key = getpass('Enter your Uplink key: ')

In [3]:
import anvil.server
import anvil.media

anvil.server.connect(uplink_key)

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [4]:
import os
import numpy as np
import joblib
import face_recognition

In [5]:
face_encodings = []
names = []
for image in os.listdir('umbrella'):
    image_path = f"umbrella/{image}"
    print(image_path)
    resident = face_recognition.load_image_file(image_path)
    face_encoding = face_recognition.face_encodings(resident)[0]
    face_encodings.append(face_encoding)
    names.append(image.split('.')[0])

joblib.dump([names, face_encodings], 'face_encodings.jl')

umbrella/Diego.jpg
umbrella/Klaus.jpg
umbrella/Sir_Reginald.jpg
umbrella/Five.jpg
umbrella/Ben.png
umbrella/Grace.png
umbrella/Allison.jpg
umbrella/Viktor.jpg
umbrella/Luther.jpg


['face_encodings.jl']

Add new resident

In [6]:
@anvil.server.callable
def add_resident(image):
    image, name = image
    try:
        with anvil.media.TempFile(image) as filename:
            visitor = face_recognition.load_image_file(filename)
        visitor_face_encodings = face_recognition.face_encodings(visitor)
        assert len(visitor_face_encodings) == 1, 'Too many faces in image.'
        visitor_face_encoding = visitor_face_encodings[0]
        names, face_encodings = joblib.load('face_encodings.jl')

        results = face_recognition.compare_faces(face_encodings, visitor_face_encoding)
        if not any(results):
            face_encodings.append(visitor_face_encoding)
            names.append(name.title())
        joblib.dump([names, face_encodings], 'face_encodings.jl')
    except Exception as e:
        print(f"Error: {e}")
        return False
    return True

In [7]:
@anvil.server.callable
def remove_resident(firstname):
    try:
        firstname = firstname.replace(' ', '_').title()
        names, face_encodings = joblib.load('face_encodings.jl')
        idx = names.index(firstname)
        del names[idx]
        del face_encodings[idx]
        joblib.dump([names, face_encodings], 'face_encodings.jl')
    except Exception as e:
        print(f"Error: {e}")
        return False
    return True

Check person(s) in image

In [8]:
@anvil.server.callable
def check_image(image):
    with anvil.media.TempFile(image) as filename:
            visitors = face_recognition.load_image_file(filename)
    visitors_face_encodings = face_recognition.face_encodings(visitors)
    names, face_encodings = joblib.load('face_encodings.jl')
    output = []
    for visitors_face_encoding in visitors_face_encodings:
        results = face_recognition.compare_faces(face_encodings, visitors_face_encoding)
        if any(results):
            person = names[np.argmax(results)]
            print(f"Welcome {person}!")
            output.append(person)
    return output, len(visitors_face_encodings) - len(output)

In [ ]:
anvil.server.wait_forever()

Welcome Five!
Welcome Ben!
Error: 'Fortune' is not in list


### Pre Anvil

In [ ]:
def check_image(image):
    visitors = face_recognition.load_image_file(image)
    visitors_face_encodings = face_recognition.face_encodings(visitors)

    names, face_encodings = joblib.load('face_encodings.jl')
    output = []
    for visitors_face_encoding in visitors_face_encodings:
        results = face_recognition.compare_faces(face_encodings, visitors_face_encoding)
        if any(results):
            person = names[np.argmax(results)]
            output.append(person)
    return output, len(visitors_face_encodings) - len(output)


In [ ]:
for unknown in os.listdir('test'):
    image_path = f"test/{unknown}"
    print(image_path)
    residents, n_visitors = check_image(image_path)

    if n_visitors > 1:
        visitors_text = f" There are {n_visitors} unknown individuals at the door."
    else:
        visitors_text = f" There is an unknown individual at the door." if n_visitors else ""

    if len(residents) > 1:
        residents_text = f"Welcome {', '.join(residents[:-1])} and {residents[-1]}."
    else:
        residents_text = f"Welcome {residents[0]}." if residents else ""

    response_text = residents_text + visitors_text
    print(response_text)

In [ ]:
def add_resident(image):
    try:
        visitor = face_recognition.load_image_file(image)
        visitor_face_encodings = face_recognition.face_encodings(visitor)
        assert len(visitor_face_encodings) == 1
        visitor_face_encoding = visitor_face_encodings[0]
        names, face_encodings = joblib.load('face_encodings.jl')

        results = face_recognition.compare_faces(face_encodings, visitor_face_encoding)
        if not any(results):
            face_encodings.append(visitor_face_encoding)
            names.append(image.split('/')[-1].split('.')[0].title())
        joblib.dump([names, face_encodings], 'face_encodings.jl')
    except Exception as e:
        print(f"Error: {e}")
        return Falsen_visitors
    return True

In [ ]:
lila = 'test/Lila.jpg'
result = check_image(lila)
if not result[0] and result[1] > 0:
    print('There is an unknown individual at the door.')
else:
    print(f"Welcome, {result[0][0]}")
print(add_resident(lila))
result = check_image(lila)
if not result[0] and result[1] > 0:
    print('There is an unknown individual at the door.')
else:
    print(f"Welcome, {result[0][0]}.")

There is an unknown individual at the door.
True
Welcome, Lila.


In [ ]:
def remove_resident(firstname):
    try:
        firstname = firstname.replace(' ', '_').title()
        names, face_encodings = joblib.load('face_encodings.jl')
        idx = names.index(firstname)
        del names[idx]
        del face_encodings[idx]
        joblib.dump([names, face_encodings], 'face_encodings.jl')
    except Exception as e:
        print(f"Error: {e}")
        return False
    return True

In [ ]:
remove_resident('Lila')
result = check_image(lila)
if not result[0] and result[1] > 0:
    print('There is an unknown individual at the door.')

There is an unknown individual at the door.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=60f840d8-661a-44b2-9944-36676f0c567f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>